In [1]:
import argparse

import pickle

import os

import scipy.stats

import numpy as np

import pandas as pd

from tabulate import tabulate

from scikit_posthocs import posthoc_conover

import sys

import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
from joblib import load
import multiprocessing
import logging

logging.basicConfig(level=logging.INFO)

In [6]:
def dev(approx, true):
    return abs(approx - true)
    
def compute_stats(dataset, alpha, log=False):

    # load results
    logging.info(f'Loading results from {dataset}...')
    with open(dataset, 'rb') as fh:
        results = pickle.load(fh)

    # analyze results
    for i, (name, r) in enumerate(results.items()):
        # compute errors
        errors = pd.DataFrame({
            h: dev(r[h], r['true_ce'])
            for h in r.head()
        })
        
        # perform ANOVA
        data = [errors[h] for h in errors.head()]
        _, anova_pvalue = scipy.stats.friedmanchisquare(*data)

        ret = (name,)
        if anova_pvalue < alpha:
            # perform post-hoc analysis
            if log:
                print("\n")
                print(f'{name}: ANOVA p-value {anova_pvalue} is significant @ {alpha}\n')

            table = []
            for h in errors.head():
                for v in errors[h]:
                    table.append(pd.DataFrame({'Metric': [h], 'Value': [v]}))
            table = pd.concat(table)
            p_values = posthoc_conover(table, p_adjust='holm', group_col='Metric', val_col='Value')

            medians = errors.median(axis=0)
            
            if log:
                print(tabulate([
                    [h, np.median(errors[h]), np.std(errors[h])] + ['diff' if abs(p) < alpha else 'same' for p in p_values[h]]
                    for h in p_values.columns
                ], headers=['Metric', 'Bias', 'Std'] + list(p_values.columns)))

            ret = name, p_values, medians

        else:
            if log:
                print(f'{name}: ANOVA p-value {anova_pvalue} is not significant @ {alpha}')

        if log:
            print("\n")

        yield ret

In [7]:
def process_single(dataset, alpha):
    with open(os.path.splitext(dataset)[0]+".log", "w") as log:
        print(f"Writing output to {log.name}")

        gen = compute_stats(dataset, alpha, log)

        for stat in gen:
            if len(stat) > 1:
                print(stat[0])
            else:
                print(f"No result for {stat[0]}")

In [36]:
def process_run(input_tuple):
    
    run, dataset, alpha = input_tuple
    
    f = os.path.join(dataset, run["filename"])
    data = []
    for ret in compute_stats(f, alpha):
        
        row = run./home/maximl/projects/reliability/notebookscopy()
        
        # if anova was significant, check median bias of ece_v{2,3} compared to ece
        if len(ret) > 1:
            name, p_values, medians = ret

            row["clf"] = name

            for m, p in p_values["ece"].drop(['true_ce', 'ece']).iteritems():

                # if ece_v{2,3} - ece is significant, check which one is more biased
                if p < alpha:

                    if medians[m] > medians["ece"]:
                        # ece is less biased 
                        row[m] = -1
                    else:
                        # ece is more biased
                        row[m] = 1
                else:
                    # similar bias
                    row[m] = 0

            data.append(row)

    return pd.DataFrame(data)

            
def process_multi(dataset, alpha):

    assert Path(dataset).is_dir(), "Dataset needs to be directory containing grid results."

    # load all meta data from runs into one dataframe
    runs = pd.concat([
        load(p) for p in
        Path(dataset).glob("*runs*")
    ]).reset_index(drop=True)

    # compute statistics for every run
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        data = pool.map(process_run, map(lambda row: (row[1], dataset, alpha), runs.iterrows()))

    data = pd.concat(data)

    with open(os.path.join(dataset, "analysis.dat"), "wb") as pkl:
        pickle.dump(data, pkl)
        
    return data

In [37]:
data = process_multi("/home/maximl/Data/Experiment_data/results/riverrel/artificial_data/899c1651a0155fcfd0bfb86b16607dde3ac19e49/", 0.05)

INFO:root:Loading results from /home/maximl/Data/Experiment_data/results/riverrel/artificial_data/899c1651a0155fcfd0bfb86b16607dde3ac19e49/107.dat...
INFO:root:Loading results from /home/maximl/Data/Experiment_data/results/riverrel/artificial_data/899c1651a0155fcfd0bfb86b16607dde3ac19e49/24.dat...
INFO:root:Loading results from /home/maximl/Data/Experiment_data/results/riverrel/artificial_data/899c1651a0155fcfd0bfb86b16607dde3ac19e49/0.dat...
INFO:root:Loading results from /home/maximl/Data/Experiment_data/results/riverrel/artificial_data/899c1651a0155fcfd0bfb86b16607dde3ac19e49/8.dat...
INFO:root:Loading results from /home/maximl/Data/Experiment_data/results/riverrel/artificial_data/899c1651a0155fcfd0bfb86b16607dde3ac19e49/16.dat...
INFO:root:Loading results from /home/maximl/Data/Experiment_data/results/riverrel/artificial_data/899c1651a0155fcfd0bfb86b16607dde3ac19e49/115.dat...
INFO:root:Loading results from /home/maximl/Data/Experiment_data/results/riverrel/artificial_data/899c1651